In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
from packages.load import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline



# 3. Shrinkage method:
## 3.1. Food:

In [2]:
hicp_all_path = 'data/preprocessed/hicp_yoy.csv'
hicp_class_path = 'data/HICP_COICOP10s.xlsx'
hicp_cat_path = 'data/preprocessed/food_yoy_infl.csv'
category = 'Food'

HICP_monthly, HICP_class, HICP_cat = import_data_all(hicp_all_path=hicp_all_path,
                                                     hicp_class_path=hicp_class_path,
                                                     hicp_cat_path=hicp_cat_path)

cat_df = split_into_category(category=category,
                             HICP_class=HICP_class,
                             HICP_monthly=HICP_monthly)

Number of items in Food group:  180


/Users/lananhnguyen/Desktop/thesis/thesis_code/main/packages/load.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df.fillna(0, inplace=True)


,Rice,Rice.preparations,Flour,Semolina..rye.flour.or.the.like,White.bread,Rye.bread.or.brown.bread,Granary.bread.or.wholemeal.bread,Ready.to.bake.rolls,Fresh.bread.rolls,Yeast.pastry,...,Pome.fruit.wine.or.wine.from.other.fruits,Wine..alcohol.content.more.than.15..,Wine.based.drinks,Pils..dark.beer.or.other.lager.beer,Wheat.beer..Altbier.or.the.like,Non.alcoholic.beer..malt.beer.or.the.like,Beer.based.drinks,Cigarettes,Cigars.and.small.cigars,Tobacco
date,,,,,,,,,,,,,,,,,,,,,
1997-01-31,-1.268741,0.000000,-0.404307,0.000000,1.805560,1.805560,2.061858,0.000000,2.090031,2.141679,...,0.000000,0.000000,0.000000,0.373129,0.500628,1.243781,0.000000,3.863638,2.941178,0.000000
1997-02-28,-1.038055,0.000000,0.406498,0.119188,1.939060,2.080444,1.762121,0.000000,1.926165,1.806237,...,0.000000,0.000000,0.000000,0.622665,0.625782,0.618047,0.000000,3.863638,2.941178,0.000000
1997-03-31,-1.153403,0.000000,0.541997,0.000000,1.798057,1.798057,1.759538,0.000000,1.762818,1.803276,...,0.000000,0.000000,0.000000,0.621891,0.750937,0.742572,0.000000,3.863638,2.941178,0.000000
1997-04-30,-0.922717,0.000000,0.541997,0.000000,1.936368,1.936368,1.608185,0.000000,1.923072,1.803276,...,0.000000,0.000000,0.000000,0.871727,0.876091,0.618047,0.000000,3.863638,2.941178,0.000000
1997-05-31,-1.153403,0.000000,0.679348,0.000000,1.793108,1.936368,1.605837,0.000000,2.083332,1.636661,...,0.000000,0.000000,0.000000,0.871727,0.876091,0.369917,0.000000,3.863638,2.941178,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,20.160570,14.652015,39.402166,28.884825,12.735423,14.559719,15.075827,22.796073,19.982245,12.357580,...,5.084750,1.757808,7.868849,7.116103,5.058723,4.775279,6.141159,4.814527,2.133335,6.503443
2022-09-30,19.383260,16.184450,39.872845,28.153560,13.879002,17.130432,13.553431,23.813793,19.930066,13.350782,...,6.019763,2.154755,9.803921,10.965328,6.733395,10.028121,5.027422,4.731861,1.676958,6.402446
2022-10-31,23.309612,17.442928,37.840289,29.055917,15.555556,17.478266,15.077991,25.823686,21.153850,14.821273,...,7.864160,3.054186,10.021783,11.131900,5.058723,10.325580,6.181821,4.320503,1.766784,5.664653


In [3]:
X_cat_train, y_cat_train = split_train_set(cat_df=cat_df,
                                           HICP_cat=HICP_cat)

Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00


In [13]:

best_param = tuning_gridsearchcv(reg= Lasso(),
                                 grid_space= np.arange(0.01, 0.2, 0.01),
                                 X_train= X_cat_train,
                                 y_train= y_cat_train,
                                 test_size=12)
best_param

{'regression__alpha': 0.03}

In [14]:
lasso = RecursiveForecast(cat_df, HICP_cat, max_horizon= 3, model=Lasso, hyperparameter=0.03)
final_forecast_df = lasso.concat_forecast()

Train test split date: 2015-12-31, model <class 'sklearn.linear_model._coordinate_descent.Lasso'>, hyperparameter: 0.03
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-12-31 00:00:00 to 2022-11-30 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00
--------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-11-30 00:00:00
Predictor test period: DatetimeIndex(['2015-11-30', '2015-12-31', '2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[2.31996332 2.07814513 1.41397014]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Predictor test period: DatetimeIndex(['2

In [15]:
cat_file_path = 'data/forecast_results/food_forecast.csv'
save_forecast('food', forecast_result_df=final_forecast_df, cat_file_path=cat_file_path)

---